In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
df = pd.read_csv('InstaCart.csv')

In [3]:
df.head()

,product_id,product_name,aisle_id,department_id,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,department,aisle
0,16617,Organic Muenster Cheese Slices,21,16,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,8.0,0.0,dairy eggs,packaged cheese
1,21267,Sourdough Bread,112,3,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,3.0,0.0,bakery,bread
2,24561,Organic Cheese Frozen Pizza,79,1,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,1.0,0.0,frozen,frozen pizza
3,24852,Banana,24,4,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,2.0,0.0,produce,fresh fruits
4,27344,Uncured Genoa Salami,96,20,2055979.0,1097.0,prior,1.0,2.0,14.0,0.0,6.0,0.0,deli,lunch meat


In [4]:
df.shape

(51613, 15)

Получаем  51613 строк и 15 столбцов.

In [5]:
df.dtypes


,0
product_id,int64
product_name,object
aisle_id,int64
department_id,int64
order_id,float64
user_id,float64
eval_set,object
order_number,float64
order_dow,float64
order_hour_of_day,float64


In [6]:
{
    'users': df['user_id'].nunique(),
    'products': df['product_id'].nunique(),
    'orders': df['order_id'].nunique()
}


{'users': 500, 'products': 9222, 'orders': 5520}

В нашем датасете 500 юзеров, 9222 различных продукта и 5520 заказов. В среднем примерно 11 заказов на каждого кастомера.

In [7]:
df.isna().sum()


,0
product_id,0
product_name,0
aisle_id,0
department_id,0
order_id,0
user_id,0
eval_set,0
order_number,0
order_dow,0
order_hour_of_day,0


Пропусков не имеем.

In [3]:
import plotly.express as px

dow_map_ru = {
    0: 'Воскресенье', 1: 'Понедельник', 2: 'Вторник', 3: 'Среда',
    4: 'Четверг', 5: 'Пятница', 6: 'Суббота'
}

df['order_dow_label'] = df['order_dow'].map(dow_map_ru)

# Счёт количества
dow_counts = df['order_dow_label'].value_counts().reset_index()
dow_counts.columns = ['День недели', 'Количество заказов']

# Порядок дней
dow_order_ru = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']
dow_counts['День недели'] = pd.Categorical(dow_counts['День недели'], categories=dow_order_ru, ordered=True)
dow_counts = dow_counts.sort_values('День недели')

fig = px.bar(
    dow_counts,
    x='День недели',
    y='Количество заказов',
    title='Распределение заказов по дням недели',
    text_auto=True,
    template='plotly_white'
)
fig.update_layout(
    xaxis_title='День недели',
    yaxis_title='Количество заказов'
)
fig.show()


**Выводы**

Пик заказов — в воскресенье, что логично: клиенты закупаются на неделю вперёд. Второй пик — понедельник, возможно, для дозакупки забытых товаров.
Минимальная активность — в среду и четверг, что делает эти дни менее приоритетными для маркетинговых акций.

Кампании и персонализированные рекомендации лучше запускать в выходные и в понедельник утром. Возможно снижение push-рассылок и затрат на рекламу в середине недели. Также, рекомендации в в воскресенье помогут клиентам не забыть купить все необходимые товары, что увеличит выручку.

In [16]:
hour_counts = df['order_hour_of_day'].value_counts().sort_index().reset_index()
hour_counts.columns = ['Час суток', 'Количество заказов']

fig = px.bar(
    hour_counts,
    x='Час суток',
    y='Количество заказов',
    title='Распределение заказов по часам суток',
    text_auto=True,
    template='plotly_white'
)
fig.update_layout(
    xaxis=dict(dtick=1),
    xaxis_title='Час суток',
    yaxis_title='Количество заказов'
)
fig.show()


**Выводы**

Клиенты чаще всего делают заказы в рабочие часы — с 10 до 16, особенно утром и после обеда. Ночное и раннее утро — мёртвое время, когда почти никто не покупает. Спад в 12:00 может быть связан с обеденным перерывом или отвлечённостью пользователей.

Персонализированные предложения и push-уведомления стоит отправлять в утренний прайм (10:00–11:00) и во второй половине дня (13:00–16:00). Нет смысла делать рассылки ночью.

In [32]:
df['order_dow_label'] = df['order_dow'].map(dow_map_ru)

# Сводная таблица
heatmap_data = df.groupby(['order_dow_label', 'order_hour_of_day'])['order_id'].count().reset_index()
heatmap_data_pivot = heatmap_data.pivot(index='order_dow_label', columns='order_hour_of_day', values='order_id')
heatmap_data_pivot = heatmap_data_pivot.reindex(dow_order_ru)

fig = go.Figure(
    data=go.Heatmap(
        z=heatmap_data_pivot.values,
        x=heatmap_data_pivot.columns,
        y=heatmap_data_pivot.index,
        colorscale='YlGnBu',
        colorbar_title='Число заказов'
    )
)
fig.update_layout(
    title='Тепловая карта активности заказов: дни недели × часы',
    xaxis_title='Час суток',
    yaxis_title='День недели',
    template='plotly_white'
)
fig.show()


**Выводы**

Заказы концентрируются вокруг выходных и офисных часов будней. Можно чётко выделить "покупательские окна" — когда клиенты наиболее восприимчивы к рекомендациям.

Эффективно таргетировать клиентов по времени недели и часу дня. Для повышения конверсии — учитывать активные окна при построении Reorder-модели (ввести фичи is_peak_time, is_weekend, hour_bucket, dow_bucket).

In [21]:
reorder_counts = df['reordered'].value_counts().reset_index()
reorder_counts.columns = ['Повторная покупка', 'Количество']
reorder_counts['Повторная покупка'] = reorder_counts['Повторная покупка'].map({0.0: 'Нет', 1.0: 'Да'})

fig = px.pie(
    reorder_counts,
    names='Повторная покупка',
    values='Количество',
    title='Доля повторных покупок',
    hole=0.3,
)
fig.show()


**Вывод**

Почти каждый второй товар в заказах — повторная покупка. Это делает задачу предсказания повторов особенно ценной: даже небольшое улучшение Reorder Score будет влиять на большую часть заказов.

In [22]:
reorder_per_order = df.groupby('order_id')['reordered'].sum().reset_index()
reorder_per_order.columns = ['order_id', 'Количество повторов']

fig = px.histogram(
    reorder_per_order,
    x='Количество повторов',
    nbins=30,
    title='Распределение количества повторных товаров в заказе',
    template='plotly_white'
)
fig.update_layout(
    xaxis_title='Количество повторных товаров в заказе',
    yaxis_title='Число заказов'
)
fig.show()


**Вывод**

Большинство заказов включают от 1 до 10 повторных товаров, а длинные заказы (>15 повторов) — относительно редки. Это можно использовать для таргетинга по сегментам: краткие vs. длинные повторы.

In [23]:
product_stats = df.groupby('product_name').agg(
    заказы=('reordered', 'count'),
    повторы=('reordered', 'sum')
).reset_index()
product_stats['доля_повторов'] = product_stats['повторы'] / product_stats['заказы']

top_repeat = product_stats[product_stats['заказы'] > 30].sort_values('доля_повторов', ascending=False).head(10)

fig = px.bar(
    top_repeat,
    x='доля_повторов',
    y='product_name',
    orientation='h',
    title='Топ-10 самых повторяемых товаров (с > 30 покупками)',
    labels={'доля_повторов': 'Доля повторов', 'product_name': 'Товар'},
    template='plotly_white'
)
fig.update_layout(yaxis=dict(tickfont=dict(size=10)))
fig.show()


**Вывод**

Существует стабильный набор «любимых» товаров, которые клиенты покупают почти всегда. Эти продукты — отличные кандидаты для персонализированных рекомендаций и автоматических подсказок («не забыть купить снова»).

In [25]:
dept_reorder = df.groupby('department').agg(
    всего=('reordered', 'count'),
    повторов=('reordered', 'sum')
).reset_index()

dept_reorder['доля_повторов'] = dept_reorder['повторов'] / dept_reorder['всего']
dept_reorder = dept_reorder.sort_values('доля_повторов', ascending=False)

fig = px.bar(
    dept_reorder,
    x='department',
    y='доля_повторов',
    title='Повторяемость заказов по категориям',
    labels={'department': 'Категория', 'доля_повторов': 'Доля повторов'},
    template='plotly_white'
)
fig.update_layout(xaxis_tickangle=45)
fig.show()


**Вывод**

Некоторые категории (например, яйца, выпечка, снеки) имеют высокую «природную» повторяемость. Это важно учитывать при построении модели — можно добавить категориальную фичу category_repeat_rate.

In [26]:
basket_size = df.groupby('order_id')['product_id'].count().reset_index()
basket_size.columns = ['order_id', 'basket_size']

fig = px.histogram(
    basket_size,
    x='basket_size',
    nbins=50,
    title='Распределение размера корзины (кол-во товаров в заказе)',
    labels={'basket_size': 'Размер корзины'},
    template='plotly_white'
)
fig.update_layout(
    xaxis_title='Количество товаров в заказе',
    yaxis_title='Число заказов'
)
fig.show()


**Вывод**

Средняя корзина — небольшая (до 10 товаров), что делает предсказание ключевых повторов критичным: 1–2 точных рекомендации могут покрыть 20–40% заказа.

In [27]:
df_filtered = df[df['add_to_cart_order'] <= 30]

fig = px.histogram(
    df_filtered[df_filtered['reordered'] == 1],
    x='add_to_cart_order',
    nbins=30,
    title='Распределение позиций добавления для повторных товаров',
    labels={'add_to_cart_order': 'Позиция в корзине'},
    template='plotly_white'
)
fig.update_layout(
    xaxis_title='Позиция в корзине (1 = первый товар)',
    yaxis_title='Количество повторов'
)
fig.show()


**Вывод**

Повторные товары часто добавляются в начале корзины. Это подтверждает гипотезу, что клиенты возвращаются за “проверенными” продуктами.
Полезная фича: avg_position_of_product, is_added_early.

In [28]:
first_items = df[df['add_to_cart_order'] == 1]
top_first = first_items['product_name'].value_counts().head(10).reset_index()
top_first.columns = ['Товар', 'Количество']

fig = px.bar(
    top_first,
    x='Количество',
    y='Товар',
    orientation='h',
    title='Топ-10 товаров, добавляемых первыми в корзину',
    template='plotly_white'
)
fig.update_layout(yaxis=dict(tickfont=dict(size=10)))
fig.show()


**Вывод**

Есть “якорные товары”, с которых клиенты начинают покупки. Эти позиции задают структуру заказа, и могут служить триггерами для рекомендаций других продуктов. Бананы 4% всех заказов.

In [29]:
intervals = df['days_since_prior_order'].dropna()

fig = px.histogram(
    intervals,
    nbins=50,
    title='Распределение интервала дней между заказами',
    labels={'value': 'Дни между заказами'},
    template='plotly_white'
)
fig.update_layout(
    xaxis_title='Дни между заказами',
    yaxis_title='Количество заказов'
)
fig.show()


**Вывод**

Многие клиенты делают заказы в тот же день (0 дней) — возможно, это отмена/повтор
7 и 30 дней — циклы повторных покупок, получаем, недельные и месячные привычки
Это отличная основа для RFM-фич и календарных предсказаний (days_since_last_order, next_order_expected_in).

In [30]:
user_order_counts = df.groupby('user_id')['order_number'].max().reset_index()
user_order_counts.columns = ['user_id', 'total_orders']

fig = px.histogram(
    user_order_counts['total_orders'],
    nbins=30,
    title='Распределение количества заказов на пользователя',
    labels={'value': 'Количество заказов'},
    template='plotly_white'
)
fig.update_layout(
    xaxis_title='Количество заказов на пользователя',
    yaxis_title='Число пользователей'
)
fig.show()


**Вывод**

Большинство пользователей — нечастые покупатели (до 10 заказов). Для таких клиентов важно дать максимально релевантные рекомендации сразу, т.к. «второго шанса» может не быть.

In [36]:
rfm['freq_bin'] = rfm['frequency'].clip(upper=50)
rfm['days_bin'] = rfm['total_days_since_first_order'].clip(upper=2000) // 100 * 100

heatmap_data = rfm.groupby(['freq_bin', 'days_bin']).size().reset_index(name='count')
pivot_table = heatmap_data.pivot(index='days_bin', columns='freq_bin', values='count').fillna(0)
pivot_table = pivot_table.sort_index(ascending=False)

fig = go.Figure(
    data=go.Heatmap(
        z=pivot_table.values,
        x=pivot_table.columns,
        y=pivot_table.index,
        colorscale='YlGnBu',
        colorbar_title='Кол-во пользователей'
    )
)

fig.update_layout(
    title='Тепловая карта: Частота заказов vs. Дней с первого заказа',
    xaxis_title='Количество заказов',
    yaxis_title='Дней с первого заказа (шаг 100)',
    template='plotly_white'
)

fig.show()


**Общий** **итог**

Покупки Instacart — цикличны по времени и категориям, с чёткими пиками.

Повторяемость высока (55.8%), а «якорные товары» открывают корзину.

Поведение пользователей устойчиво и предсказуемо, особенно в рамках повторов и времени покупок.

Что это даёт бизнесу:

Возможность персонализировать рекомендации на основе временных паттернов и истории.

Повысить конверсию в «добавить в корзину» и средний чек.

Снизить затраты на бесполезные коммуникации, сосредоточившись на ключевых пользователях и пиках активности.

In [8]:
cat_hour = df.groupby(['department', 'order_hour_of_day']).size().reset_index(name='count')
pivot_hour = cat_hour.pivot(index='department', columns='order_hour_of_day', values='count').fillna(0)

fig = go.Figure(
    data=go.Heatmap(
        z=pivot_hour.values,
        x=pivot_hour.columns,
        y=pivot_hour.index,
        colorscale='Blues',
        colorbar_title='Кол-во товаров'
    )
)

fig.update_layout(
    title='Популярность категорий по часам суток',
    xaxis_title='Час суток',
    yaxis_title='Категория',
    template='plotly_white'
)
fig.show()


Явный лидер — категория produce (овощи, фрукты)
Эта категория стабильно востребована на протяжении всего дня
Также хорошо выделяются: dairy eggs.

beverages и pantry — тоже неплохо